In [1]:
import numpy as np
import os
import collections
import scipy.io as io
from scipy.stats import bernoulli
import datetime
import matplotlib.pyplot as plt
from functools import reduce
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn import metrics
# import xgboost as xgb
# from sklearn.preprocerssing import OneHotEncoder
from scipy.stats import bernoulli
from sklearn.metrics import classification_report, precision_recall_fscore_support
from torch.utils import data as Data
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import json
import gc
%matplotlib inline

In [2]:
sensor = np.loadtxt('./data/processed_data/sensor_data.txt')
activity = np.loadtxt('./data/processed_data/activity_data.txt')

In [3]:
class Denoising_AutoEncoder(nn.Module):
    def __init__(self, n_input, n_hidden):
        super(Denoising_AutoEncoder, self).__init__()
        self.n_input = n_input
        self.encoder = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.Sigmoid()
#             nn.SELU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(n_hidden, n_input),
            nn.Sigmoid()
#             nn.SELU()
        )
        
    def forward(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return encoded, decoded

## 应该生成服从伯努利分布的随机噪声
def make_noise(shape):
    noise = bernoulli.rvs(0.1, size=shape)
#     mask = np.random.random(size=shape)
#     mask = (mask >= 0.8).astype(int)
#     noise = noise * mask
    return noise

def init_weights(m):
    if type(m) == nn.Linear:
        m.weight.data.fill_(0.02)
#     print(m.weight)

#     net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
#     net.apply(init_weights)

def train_d(net, learning_rate, criterion, optimizer, train_x, max_iter):
    all_loss = []
    loss = None
    for i in range(max_iter):
        optimizer.zero_grad()
#         noise = Variable(torch.FloatTensor(make_noise(train_en_time_X.shape)))
#         encoded, decoded = net(train_x + noise)
        noise = make_noise(train_x.shape)
        encoded, decoded = net(Variable(torch.FloatTensor(np.logical_or(train_x, noise))))
#         labels = decoded
#         target = Variable(torch.LongTensor(train_x))
#         print(labels.shape, target.shape, labels.view(1, -1).shape, target.view(1, -1).shape)
        loss = criterion(decoded, Variable(torch.FloatTensor(train_x)))
        loss.backward(retain_variables=True)
        optimizer.step()
        if i % 2 == 0:
            all_loss.append(loss)
#             print('epoch: '+str(i)+' loss: '+str(loss))
            print('\r\t Train Epoch: {}\tLoss: {:.6f}'.format(
                i, 
                loss.cpu().data[0]), end='')
        if i % 20 == 0:
            print()
    return encoded, all_loss

In [4]:
learning_rate = 0.0001

criterion = nn.MSELoss(size_average=False)
model = Denoising_AutoEncoder(14, 1)
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate,
                             betas=(0.5, 0.999))
# optimizer = torch.optim.SGD(model.parameters(),
#                            lr = learning_rate)
encoded, all_loss = train_d(model, learning_rate, criterion,
                          optimizer, sensor, 45)
## Classification Model Training
features = encoded.data.numpy()

D:\Anaconda3_501\lib\site-packages\torch\autograd\__init__.py:93: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


	 Train Epoch: 0	Loss: 279101.218750
	 Train Epoch: 20	Loss: 271556.468750
	 Train Epoch: 40	Loss: 263801.281250
	 Train Epoch: 44	Loss: 263526.843750

In [30]:
features.shape

(80010, 30)

day = int((60 * 60 * 24) / 30)
X = []
y = []

i = 0
while i < new_sensor.shape[0]:
    X_test = new_sensor[i: i+day, :]
    y_test = new_act[i: i+day, :]
    X_train = np.vstack((new_sensor[:i, :], new_sensor[i+day:, :]))
    y_train = np.vstack((new_act[:i, :], new_act[i+day:, :]))
    X.append((X_train, X_test))
    y.append((y_train, y_test))
    i += day

###完成对于特征的重组

In [ ]:
learning_rate = 0.0002

precisions = []
recalls = []
f1s = []
accuracys = []

for i in range(len(X)):
    ## Feature Representation
    print('\nTraining {}th fold:'.format(i))
    X_train, X_test = np.asarray(X[i][0]), np.asarray(X[i][1])
    y_train, y_test = np.asarray(y[i][0]), np.asarray(y[i][1])
#     criterion = nn.MSELoss(size_average=True)
#     criterion = nn.NLLLoss2d()
    criterion = nn.MSELoss()
    model = Denoising_AutoEncoder(42, 60)
    model.apply(init_weights)
    optimizer = torch.optim.RMSprop(model.parameters(),
                                    lr=learning_rate)
    encoded, all_loss = train(model, learning_rate, criterion,
                              optimizer, X_train, 30)
    ## Classification Model Training
    features = encoded.data.numpy()
    y_preds = []
    for i in range(y_train.shape[1]):
#         dt = DecisionTreeClassifier()
#         dt.fit(X_train, y_train[:, i])
#         y_preds.append(dt.predict(X_test))
        rf = RandomForestClassifier(n_estimators=25, max_depth=3, max_features=3)
        rf.fit(X_train, y_train[:, i])
        y_preds.append(rf.predict(X_test))
    y_preds = np.asarray(y_preds).T
#     print(y_preds.shape)
    
    accuracys.append(metrics.accuracy_score(y_test, y_preds))
    precisions.append(metrics.precision_score(y_test, y_preds, average='micro'))
    recalls.append(metrics.recall_score(y_test, y_preds, average='micro'))
    f1s.append(metrics.f1_score(y_test, y_preds, average='micro'))

print('Accuracy: {} | Precision: {} | Recall: {} | F1: {}'.format(np.mean(accuracys), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

### 第三种方法

#### 尝试使用一种类似于Word2Vec的编码方法编码前后时间信息

### 这里由于压缩的是另一个维度，意思是假设有n条数据，每条数据有m个特征，之前压缩的是m个特征，但是这里压缩的是n。

In [3]:
class Encoder(nn.Module):
    def __init__(self, n_input, n_hidden):
        super(Encoder, self).__init__()
        self.n_input = n_input
        self.encoder = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(n_hidden, n_input),
            nn.Sigmoid()
        )
        
    def forward(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return encoded, decoded

def make_noise(shape):
    noise = bernoulli.rvs(0.1, size=shape)
#     mask = np.random.random(size=shape)
#     mask = (mask >= 0.8).astype(int)
#     noise = noise * mask
    return noise

def init_weights(m):
    if type(m) == nn.Linear:
        m.weight.data.fill_(0.02)

def train(net, learning_rate, criterion, optimizer, train_x, max_iter):
    all_loss = []
    loss = None
    for i in range(max_iter):
        optimizer.zero_grad()
        noise = make_noise(train_x.shape)
        encoded, decoded = net(Variable(torch.FloatTensor(np.logical_or(train_x, noise))))
#         encoded, decoded = net(Variable(torch.FloatTensor(train_x)))
#         labels = decoded
#         target = Variable(torch.LongTensor(train_x))
#         print(labels.shape, target.shape, labels.view(1, -1).shape, target.view(1, -1).shape)
        loss = criterion(decoded, Variable(torch.FloatTensor(train_x)))
        loss.backward(retain_variables=True)
        optimizer.step()
        if i % 2 == 0:
            all_loss.append(loss)
#             print('epoch: '+str(i)+' loss: '+str(loss))
            print('\r\t Train Epoch: {}\tLoss: {:.6f}'.format(
                i, 
                loss.cpu().data[0]))

    return encoded, all_loss

In [4]:
sensor.shape

(80010, 14)

In [9]:
# ## 划分窗口大小
# window_size = 7

# # print(sensor.shape, (sensor.shape[0], padding))
# padded_activity = np.hstack((np.zeros((sensor.shape[0], padding)),
#                              np.hstack((sensor, np.zeros((sensor.shape[0], padding))))))
# # print(sensor.shape, padded_activity.shape)
# sensor_feature = np.zeros(((window_size - 1) * sensor.shape[0], sensor.shape[1]))
# print(sensor_feature.shape)
# for i in range(padding, padding + sensor_feature.shape[1]):
# #     print(padded_activity[:, (i-(window_size//2)):i].T.shape,
# #           padded_activity[:, (i+1):(i+(window_size//2)+1)].T.shape)
#     sensor_feature[:, i-padding] = np.vstack((padded_activity[:, (i-(window_size//2)):i].T,
#                                       padded_activity[:, (i+1):(i+(window_size//2)+1)].T)).reshape(-1)

In [10]:
## 划分窗口数据
window_size = 16

window_data = np.zeros((window_size * sensor.shape[1], sensor.shape[0] // window_size))

for i in range(window_data.shape[1]):
    window_data[:, i] = sensor[i * window_size:(i + 1) * window_size, :].ravel()
window_data = window_data.T
window_data.shape

(5000, 224)

In [11]:
learning_rate = 0.001

criterion = nn.MSELoss()
model = Encoder(window_data.shape[1], 28)
model.apply(init_weights)
optimizer = torch.optim.RMSprop(model.parameters(),
                                lr=learning_rate)
encoded, all_loss = train(model, learning_rate, criterion,
                          optimizer, window_data, 5)
## Classification Model Training
new_features = encoded.data.numpy()

D:\Anaconda3_501\lib\site-packages\torch\autograd\__init__.py:93: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


	 Train Epoch: 0	Loss: 0.252957
	 Train Epoch: 2	Loss: 0.248041
	 Train Epoch: 4	Loss: 0.245677


In [22]:
f = 0
for i in range(5000):
    if (new_features[i, :] != new_features[0, :]).any():
        f += 1

In [24]:
from sklearn.naive_bayes import GaussianNB

In [25]:
day = int(60 * 60 * 24 / 30)
l = sensor.shape[0] // day
# print(l)
precisions = []
recalls = []
f1s = []
accuracys = []

for i in range(l):
    ## Feature Representation
    print('\nTraining {}th fold:'.format(i))
    X_test = new_features[i*day:(i+1)*day, :]
    X_train = np.vstack((new_features[:i*day, :], new_features[(i+1)*day:, :]))
    y_test = activity[i*day:(i+1)*day, :]
    y_train = np.vstack((activity[:i*day, :], activity[(i+1)*day:, :]))  
    y_preds = []
    for i in range(y_train.shape[1]):
        rf = RandomForestClassifier(n_estimators=50, max_depth=3, max_features=3)
        rf.fit(X_train, y_train[:, i])
        y_pred = rf.predict(X_test)
        y_preds.append(y_pred) 
    y_preds = np.asarray(y_preds).T    
    
    accuracy = metrics.accuracy_score(y_test, y_preds)
    precision = metrics.precision_score(y_test, y_preds, average='micro')
    recall = metrics.recall_score(y_test, y_preds, average='micro')
    f1 = metrics.f1_score(y_test, y_preds, average='micro')
    print('\tAccuracy: {} | Precision: {} | Recall: {} | F1: {}'.format(accuracy,
                                                                        precision,
                                                                        recall,
                                                                        f1))
    
    accuracys.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)


Training 0th fold:


ValueError: Number of labels=77130 does not match number of samples=2120

In [ ]:
print('Accuracy: {} | Precision: {} | Recall: {} | F1: {}'.format(np.mean(accuracys),
                                                                  np.mean(precisions),
                                                                  np.mean(recalls),
                                                                  np.mean(f1s)))

In [10]:
print('Accuracy: {} | Precision: {} | Recall: {} | F1: {}'.format(np.mean(accuracys),
                                                                  np.mean(precisions),
                                                                  np.mean(recalls),
                                                                  np.mean(f1s)))

Accuracy: 0.7601851851851853 | Precision: 0.7703532657248031 | Recall: 0.872567805642764 | F1: 0.8149614643342248


In [9]:
?xgb.XGBClassifier

做其他算法的precision-recall用作对比

In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [ ]:
nb = GaussianNB()
nb.fit(sensor, activity)
nb_pred = nb.pre